# GregoriOCR

In [1]:
#Libraries
from keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential
import numpy as np

0. Preprocessing

    a. Download GregoBase data

    b. Preprocess data

        i. Use height of staves to scale image.

        ii. Extract gabc notes from training data?

In [ ]:
#METHOD FOR PROFESSOR TO ACCESS THE DATASET

#Download the dataset: (This is currently set for another project's dataset.)
import platform
mysystem = platform.system()
file_id = '10PSeKeL3aUA56faRhr4ZfkEPcVtKjlry'
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id
# Check if system is Windows
if mysystem != 'Windows':
    !wget -O dataset.csv --no-check-certificate "$file_download_link"
    # !unzip data.zip

print('Please download the data using the following link:', file_download_link)

1. OCR

    a. Document Layout Analysis: Scanning function finds notes and puts them into individual boxes of uniform size.

In [ ]:
#FUNCTION THAT DETERMINES WHTHER SCANNING WINDOW CONTAINS A NOTE

def is_note():
    is_note = False
    ...
    
    return is_note

In [ ]:
#WINDOW THAT SCANS IMAGE FOR NOTES

#Crop a subset of an image (about the height of a staff and about the width of a note).
#Use function to determine whether this is a note.
#If it is a note:
    #Add the image to a list
    #Move window to the right by the width of one note
#Else:
    #Move the window one pixel to the right

def scan_window():
    notes = []
    ...
    
    return notes

    b. Keras CNN to perform OCR

In [ ]:
#LOADING THE DATASET

#This is taken from my ECS 171 homework assignment that uses neural networks to determine whther an image is or isn't a cat.
#Those images had been converted to numpy (.npz) files beforehand.
#Who knows if we'll do that, but it's nice for a general idea.

data = np.load('./data/cats/cats.npz') #This is set for a different dataset

#For looking at the file contents
lst = data.files
for item in lst:
    print(item)
    print(data[item])


X_train, y_train = data['Xtrain'].transpose(), data['Ytrain'].transpose()
X_test, y_test = data['Xtest'].transpose(), data['Ytest'].transpose()

#Save original shapes of X_train and X_test in case we want them later.
X_train_orig = X_train
X_test_orig = X_test

#Reshape the data from a 1D vector to a 2D matrix.
#This represents a grid of pixel coordinates.
#X_train.shape[0] returns our number of observations in X_train.
#Parameters of .reshape():
    #.reshape([number of observations in dataset, height in pixels, width in pixels, number of color dimensions])
X_train = X_train.reshape([X_train.shape[0], 64, 64, 1])


display(X_train.shape)
display(y_train.shape)
display(X_test.shape)
display(y_test.shape)

In [ ]:
#ONE-HOT ENCODE y_train

#Use one-hot encoding to transform y_train into a matrix where each column represents a different possible outcome.
#This allows the model to predict more than 2 classes.
#In our case, y_train will be m x n, where m is the number of observations and n is the number of possible notes/outcomes.

y_train_single = y_train
one_hot_encoding = pd.get_dummies(y_train)
y_train = one_hot_encoding

display(y_train)

In [ ]:
#CONVOLUSIONAL NEURAL NETWORK

model = Sequential() #Initialize the model.

#LAYERS:
#Convulusional layers:
#Supposedly selu is better than relu.  It's worth testing.
model.add(Conv2D(32, (3, 3), activation='selu'))
model.add(Conv2D(32, (3, 3), activation='selu'))

#Always flatten inputs after you convulusional layers are finished:
model.add(Flatten())

#Non-convolusional layers:
model.add(Dense(units=30, activation='selu', input_dim=X_train.shape[1]))
model.add(Dense(units=15, activation='selu'))

#Output Layer:
#n := number of possible predictions/notes (88?)
model.add(Dense(units=n, activation='softmax'))

In [ ]:
#COMPILE THE MODEL

#ECS 171 professor says adam is awesome optimizer.
#We use categorical rather than binary cross entropy because we are predicting more than 2 classes.
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
#TRAIN THE MODEL

model.fit(X_train.astype('float'), y_train, batch_size=1, epochs=10)

In [ ]:
#PREDICT TEST DATA

yhat_test = model.predict(X_test.astype(float))

#The output will be n values.
#We take the outcome associated with the the largest value as our result/prediction.
yhat = []
for y in yhat_test:
    yhat.append(np.argmax(y)) #Outcome associated with the largest value.
yhat = np.array(yhat)

print(yhat.shape)

2. Accuracy Evaluation Function

Compare our NN's output with the gabc file from GregoBase.
Note: In gabc files, the notes are within parentheses, so we can simply pick them out.
Our output will be a list of notes represented as strings.

The accuracy function simply compares our list with the notes present in the gabc.
There are many ways to do this comparison.

The simplest is to compare the first note in our list with the first note in the gabc, then the second note in our list with the second note in the gabc, and so on.
The flaw with this is that if our scanning window misses a note, our whole accuracy measure will be thrown off.

Another way to do this is to check the quantity of each type of note in the original gabc file and see if it matches the quantity in our own output.
E.g., there are 12 g notes in the original, did our neural net find 12 g notes?

In [ ]:
def accuracy():

3. Gregorio Format: Output predictions as a gabc file

Pretty simple, write each string/note in our output list to a gabc file.
Also, enclose each note in parentheses.

In [ ]:
def to_gabc():